In [14]:
# Input data files are available in the "../input/" directory.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import matplotlib.pyplot as plt
# %matplotlib inline

# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# import seaborn as sns
import string
# from wordcloud import STOPWORDS
from collections import defaultdict
import re
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
# from nltk.stem import PorterStemmer
# from sqlalchemy import create_engine

# psql_engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
#     'skomissarov',
#     '123',
#     'localhost',
#     '5432',
#     'ml_vs_covid'
# ))


In [20]:
r = psql_engine.execute('''select * from hackathon_v2.cash_response''')

In [15]:
#  title, text, label

# for 1 - FaKE
# for 0 - TRUE

news1 = pd.read_csv('train.csv')
news1.drop(['id', 'author'], axis=1, inplace=True)

news2 = pd.read_csv('BuzzFeed_fake_news_content.csv')
news2_fake = news2[['title','text']]
news2_fake['label'] = 1
news2 = pd.read_csv('BuzzFeed_real_news_content.csv')
news2_real = news2[['title','text']]
news2_real['label'] = 0
news2 = pd.concat([news2_fake, news2_real])

news3 = pd.read_csv('fake_or_real_news.csv').drop('Unnamed: 0', axis=1)
news3['label'] = news3['label'].replace({'FAKE':1, 'REAL':0})

news4 = pd.read_csv('PolitiFact_fake_news_content.csv')
news4_fake = news4[['title','text']]
news4 = pd.read_csv('PolitiFact_real_news_content.csv')
news4_real = news4[['title','text']]

news4_fake['label'] = 1
news4_real['label'] = 0
news4 = pd.concat([news4_fake, news4_real])

news5 = pd.read_csv('data.csv')
news5.drop('URLs', axis=1, inplace=True)
news5['label'] = news5['Label'].replace({0:1, 1:0})
news5['title'] = news5['Headline']
news5['text'] =  news5['Body']
news5 = news5[['title','text','label']]

news = pd.concat([news1, news2, news3, news4, news5])
df = news.reset_index().drop('index', axis=1)
df['full'] = df['']

/usr/local/anaconda3/envs/ml_vs_covid/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda3/envs/ml_vs_covid/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/envs/ml_vs_covid/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [3]:
df.drop([47, 229, 499,774,1191,1291, 1600, 1645, 1692,1702,1852,1859, 2361,2754,2758,2887], inplace=True)
df.fillna('', inplace=True)

In [4]:
y = df['label'] 
X_title = df['title']
X_text = df['text']

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
import scipy
import pickle

title_clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', ngram_range=(2, 3), max_features=100000)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', ngram_range=(2, 3), max_features=10000000)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

In [7]:
%%time
title_clf.fit(X_title_cleaned, y)
with open('models/title_clf.pickle', 'wb') as f:
    pickle.dump(title_clf, f)
accuracy_score(title_clf.predict(X_title_cleaned), y)

CPU times: user 3.26 s, sys: 179 ms, total: 3.44 s
Wall time: 3.6 s


0.7020285261489699

In [8]:
%%time
text_clf.fit(X_text_cleaned, y)
accuracy_score(text_clf.predict(X_text_cleaned), y)
with open('models/text_clf.pickle', 'wb') as f:
    pickle.dump(text_clf, f)

CPU times: user 5min 13s, sys: 49 s, total: 6min 2s
Wall time: 7min 17s


In [ ]:
import pickle

class Model:
    
    def __init__(self):
        with open('models/title_clf.pickle', 'rb') as f:
            self.title_clf = pickle.load(f)
        with open('models/text_clf.pickle', 'rb') as f:
            self.text_clf = pickle.load(f)
            
    def predict(data):
        title_cleaned = clean(data['title'])
        text_cleaned = clean(data['text'])

        predict_title = self.title_clf([title_cleaned])[0]
        predict_text = self.text_clf([text_cleaned])[0]
        
        return predict_title, predict_text
    
    def clean(self, tex): 
        tex = tex.lower()
        #clear at all

        # Urls
        tex = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tex)
        tex = re.sub(r"http?:\/\/t.co\/[A-Za-z0-9]+", "", tex)

        # Words with punctuations and special characters
        punctuations = '@#!?+&*[]-%.—:,/();$=><|{}^' + '\"\'\’\‘\“\”'
        for p in punctuations:
            tex = tex.replace(p, ' ')

        # ... and ..
        tex = tex.replace('...', ' ... ')
        if '...' not in tex:
            tex = tex.replace('..', ' ... ')      

        tex = re.sub('','',tex)
        tex = re.sub(' didn ’ t ',' did not ',tex)
        tex = re.sub(' don t ',' do not ',tex)
        tex = re.sub('&',' and ',tex)

        tex = re.sub(' $ ',' dollar ',tex)
        tex = re.sub(' fbi ', ' federal bureau of investigation ', tex)
        tex = re.sub('w /','',tex)
        tex = re.sub('\"\'\’\‘\‘', '\'', tex)
        tex = re.sub(' u . s . ',' united states ',tex)
        tex = re.sub(' u s ',' united states ',tex)
        tex = re.sub(' m . i . t . ',' massachusetts institute of technology ',tex)
        tex = re.sub(' mit ',' massachusetts institute of technology ',tex)
        tex = re.sub(' n . y . u ',' new york university ',tex)
        tex = re.sub(' nyu ',' new york university ',tex)
        tex = re.sub(' eu ',' european union ',tex)
        tex = re.sub(' e . u . ',' european union ',tex)
        tex = re.sub('>','',tex)
        tex = re.sub('<','',tex)
        tex = re.sub('%',' percent ',tex)
        tex = re.sub(' nyc ',' new york city ',tex)
        tex = re.sub(' s ',' ',tex)

        tex = re.sub(' january ',' MONTH ',tex)
        tex = re.sub(' february ',' MONTH ',tex)
        tex = re.sub(' march ',' MONTH ',tex)
        tex = re.sub(' april ',' MONTH ',tex)
        tex = re.sub(' may ',' MONTH ',tex)
        tex = re.sub(' june ',' MONTH ',tex)
        tex = re.sub(' july ',' MONTH ',tex)
        tex = re.sub(' august ',' MONTH ',tex)
        tex = re.sub(' september ',' MONTH ',tex)
        tex = re.sub(' october ',' MONTH ',tex)
        tex = re.sub(' november ',' MONTH ',tex)
        tex = re.sub(' december ',' MONTH ',tex)

        tex = re.sub(' the ',' ',tex)
        tex = re.sub(' a ',' ',tex)
        tex = re.sub(' t ',' ',tex)
        tex = re.sub(' to ',' ',tex)
        tex = re.sub(' of ',' ',tex)
        tex = re.sub(' be ',' ',tex)
        tex = re.sub(' in ',' ',tex)

        tex = re.sub('  ',' ',tex)
        tex = re.sub('  ',' ',tex)
        tex = re.sub('  ',' ',tex)

        l = []
        for i in tex.split():
            if i.isdigit():
                l.append('NUM')
            else:
                l.append(i)

        tex = ' '.join(l)

        tex = re.sub(' MONTH NUM NUM ',' DATE ',tex)
        tex = re.sub(' NUM NUM NUM ',' DATE ',tex)

        return tex


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
import scipy
import pickle

In [4]:
with open('models/title_clf.pickle', 'rb') as f:
    title_clf = pickle.load(f)
with open('models/text_clf.pickle', 'rb') as f:
    text_clf = pickle.load(f)

/usr/local/anaconda3/envs/ml_vs_covid/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/anaconda3/envs/ml_vs_covid/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/anaconda3/envs/ml_vs_covid/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.stochastic_gradient module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  war

In [9]:
title_clf.predict(['ajfgnaojngjoaong'])[0]

1

## new model

In [44]:
# coding=utf-8
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Tokenization classes implementation.

The file is forked from:
https://github.com/google-research/bert/blob/master/tokenization.py.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import re
import unicodedata

import six
import tensorflow as tf

import sentencepiece as spm

SPIECE_UNDERLINE = "▁"


def validate_case_matches_checkpoint(do_lower_case, init_checkpoint):
  """Checks whether the casing config is consistent with the checkpoint name."""

  # The casing has to be passed in by the user and there is no explicit check
  # as to whether it matches the checkpoint. The casing information probably
  # should have been stored in the bert_config.json file, but it's not, so
  # we have to heuristically detect it to validate.

  if not init_checkpoint:
    return

  m = re.match("^.*?([A-Za-z0-9_-]+)/bert_model.ckpt", init_checkpoint)
  if m is None:
    return

  model_name = m.group(1)

  lower_models = [
      "uncased_L-24_H-1024_A-16", "uncased_L-12_H-768_A-12",
      "multilingual_L-12_H-768_A-12", "chinese_L-12_H-768_A-12"
  ]

  cased_models = [
      "cased_L-12_H-768_A-12", "cased_L-24_H-1024_A-16",
      "multi_cased_L-12_H-768_A-12"
  ]

  is_bad_config = False
  if model_name in lower_models and not do_lower_case:
    is_bad_config = True
    actual_flag = "False"
    case_name = "lowercased"
    opposite_flag = "True"

  if model_name in cased_models and do_lower_case:
    is_bad_config = True
    actual_flag = "True"
    case_name = "cased"
    opposite_flag = "False"

  if is_bad_config:
    raise ValueError(
        "You passed in `--do_lower_case=%s` with `--init_checkpoint=%s`. "
        "However, `%s` seems to be a %s model, so you "
        "should pass in `--do_lower_case=%s` so that the fine-tuning matches "
        "how the model was pre-training. If this error is wrong, please "
        "just comment out this check." %
        (actual_flag, init_checkpoint, model_name, case_name, opposite_flag))


def convert_to_unicode(text):
  """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text.decode("utf-8", "ignore")
    elif isinstance(text, unicode):
      return text
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def printable_text(text):
  """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text
    elif isinstance(text, unicode):
      return text.encode("utf-8")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def load_vocab(vocab_file):
  """Loads a vocabulary file into a dictionary."""
  vocab = collections.OrderedDict()
  index = 0
  with tf.io.gfile.GFile(vocab_file, "r") as reader:
    while True:
      token = convert_to_unicode(reader.readline())
      if not token:
        break
      token = token.strip()
      vocab[token] = index
      index += 1
  return vocab


def convert_by_vocab(vocab, items):
  """Converts a sequence of [tokens|ids] using the vocab."""
  output = []
  for item in items:
    output.append(vocab[item])
  return output


def convert_tokens_to_ids(vocab, tokens):
  return convert_by_vocab(vocab, tokens)


def convert_ids_to_tokens(inv_vocab, ids):
  return convert_by_vocab(inv_vocab, ids)


def whitespace_tokenize(text):
  """Runs basic whitespace cleaning and splitting on a piece of text."""
  text = text.strip()
  if not text:
    return []
  tokens = text.split()
  return tokens


class FullTokenizer(object):
  """Runs end-to-end tokenziation."""

  def __init__(self, vocab_file, do_lower_case=True, split_on_punc=True):
    self.vocab = load_vocab(vocab_file)
    self.inv_vocab = {v: k for k, v in self.vocab.items()}
    self.basic_tokenizer = BasicTokenizer(
        do_lower_case=do_lower_case, split_on_punc=split_on_punc)
    self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    return split_tokens

  def convert_tokens_to_ids(self, tokens):
    return convert_by_vocab(self.vocab, tokens)

  def convert_ids_to_tokens(self, ids):
    return convert_by_vocab(self.inv_vocab, ids)


class BasicTokenizer(object):
  """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

  def __init__(self, do_lower_case=True, split_on_punc=True):
    """Constructs a BasicTokenizer.

    Args:
      do_lower_case: Whether to lower case the input.
      split_on_punc: Whether to apply split on punctuations. By default BERT
        starts a new token for punctuations. This makes detokenization difficult
        for tasks like seq2seq decoding.
    """
    self.do_lower_case = do_lower_case
    self.split_on_punc = split_on_punc

  def tokenize(self, text):
    """Tokenizes a piece of text."""
    text = convert_to_unicode(text)
    text = self._clean_text(text)

    # This was added on November 1st, 2018 for the multilingual and Chinese
    # models. This is also applied to the English models now, but it doesn't
    # matter since the English models were not trained on any Chinese data
    # and generally don't have any Chinese data in them (there are Chinese
    # characters in the vocabulary because Wikipedia does have some Chinese
    # words in the English Wikipedia.).
    text = self._tokenize_chinese_chars(text)

    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
      if self.do_lower_case:
        token = token.lower()
        token = self._run_strip_accents(token)
      if self.split_on_punc:
        split_tokens.extend(self._run_split_on_punc(token))
      else:
        split_tokens.append(token)

    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    return output_tokens

  def _run_strip_accents(self, text):
    """Strips accents from a piece of text."""
    text = unicodedata.normalize("NFD", text)
    output = []
    for char in text:
      cat = unicodedata.category(char)
      if cat == "Mn":
        continue
      output.append(char)
    return "".join(output)

  def _run_split_on_punc(self, text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
      char = chars[i]
      if _is_punctuation(char):
        output.append([char])
        start_new_word = True
      else:
        if start_new_word:
          output.append([])
        start_new_word = False
        output[-1].append(char)
      i += 1

    return ["".join(x) for x in output]

  def _tokenize_chinese_chars(self, text):
    """Adds whitespace around any CJK character."""
    output = []
    for char in text:
      cp = ord(char)
      if self._is_chinese_char(cp):
        output.append(" ")
        output.append(char)
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)

  def _is_chinese_char(self, cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
        (cp >= 0x3400 and cp <= 0x4DBF) or  #
        (cp >= 0x20000 and cp <= 0x2A6DF) or  #
        (cp >= 0x2A700 and cp <= 0x2B73F) or  #
        (cp >= 0x2B740 and cp <= 0x2B81F) or  #
        (cp >= 0x2B820 and cp <= 0x2CEAF) or
        (cp >= 0xF900 and cp <= 0xFAFF) or  #
        (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
      return True

    return False

  def _clean_text(self, text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
      cp = ord(char)
      if cp == 0 or cp == 0xfffd or _is_control(char):
        continue
      if _is_whitespace(char):
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)


class WordpieceTokenizer(object):
  """Runs WordPiece tokenziation."""

  def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=200):
    self.vocab = vocab
    self.unk_token = unk_token
    self.max_input_chars_per_word = max_input_chars_per_word

  def tokenize(self, text):
    """Tokenizes a piece of text into its word pieces.

    This uses a greedy longest-match-first algorithm to perform tokenization
    using the given vocabulary.

    For example:
      input = "unaffable"
      output = ["un", "##aff", "##able"]

    Args:
      text: A single token or whitespace separated tokens. This should have
        already been passed through `BasicTokenizer.

    Returns:
      A list of wordpiece tokens.
    """

    text = convert_to_unicode(text)

    output_tokens = []
    for token in whitespace_tokenize(text):
      chars = list(token)
      if len(chars) > self.max_input_chars_per_word:
        output_tokens.append(self.unk_token)
        continue

      is_bad = False
      start = 0
      sub_tokens = []
      while start < len(chars):
        end = len(chars)
        cur_substr = None
        while start < end:
          substr = "".join(chars[start:end])
          if start > 0:
            substr = "##" + substr
          if substr in self.vocab:
            cur_substr = substr
            break
          end -= 1
        if cur_substr is None:
          is_bad = True
          break
        sub_tokens.append(cur_substr)
        start = end

      if is_bad:
        output_tokens.append(self.unk_token)
      else:
        output_tokens.extend(sub_tokens)
    return output_tokens


def _is_whitespace(char):
  """Checks whether `chars` is a whitespace character."""
  # \t, \n, and \r are technically control characters but we treat them
  # as whitespace since they are generally considered as such.
  if char == " " or char == "\t" or char == "\n" or char == "\r":
    return True
  cat = unicodedata.category(char)
  if cat == "Zs":
    return True
  return False


def _is_control(char):
  """Checks whether `chars` is a control character."""
  # These are technically control characters but we count them as whitespace
  # characters.
  if char == "\t" or char == "\n" or char == "\r":
    return False
  cat = unicodedata.category(char)
  if cat in ("Cc", "Cf"):
    return True
  return False


def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False


def preprocess_text(inputs, remove_space=True, lower=False):
  """Preprocesses data by removing extra space and normalize data.

  This method is used together with sentence piece tokenizer and is forked from:
  https://github.com/google-research/google-research/blob/master/albert/tokenization.py

  Args:
    inputs: The input text.
    remove_space: Whether to remove the extra space.
    lower: Whether to lowercase the text.

  Returns:
    The preprocessed text.

  """
  outputs = inputs
  if remove_space:
    outputs = " ".join(inputs.strip().split())

  if six.PY2 and isinstance(outputs, str):
    try:
      outputs = six.ensure_text(outputs, "utf-8")
    except UnicodeDecodeError:
      outputs = six.ensure_text(outputs, "latin-1")

  outputs = unicodedata.normalize("NFKD", outputs)
  outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
  if lower:
    outputs = outputs.lower()

  return outputs


def encode_pieces(sp_model, text, sample=False):
  """Segements text into pieces.

  This method is used together with sentence piece tokenizer and is forked from:
  https://github.com/google-research/google-research/blob/master/albert/tokenization.py


  Args:
    sp_model: A spm.SentencePieceProcessor object.
    text: The input text to be segemented.
    sample: Whether to randomly sample a segmentation output or return a
      deterministic one.

  Returns:
    A list of token pieces.
  """
  if six.PY2 and isinstance(text, six.text_type):
    text = six.ensure_binary(text, "utf-8")

  if not sample:
    pieces = sp_model.EncodeAsPieces(text)
  else:
    pieces = sp_model.SampleEncodeAsPieces(text, 64, 0.1)
  new_pieces = []
  for piece in pieces:
    piece = printable_text(piece)
    if len(piece) > 1 and piece[-1] == "," and piece[-2].isdigit():
      cur_pieces = sp_model.EncodeAsPieces(piece[:-1].replace(
          SPIECE_UNDERLINE, ""))
      if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
        if len(cur_pieces[0]) == 1:
          cur_pieces = cur_pieces[1:]
        else:
          cur_pieces[0] = cur_pieces[0][1:]
      cur_pieces.append(piece[-1])
      new_pieces.extend(cur_pieces)
    else:
      new_pieces.append(piece)

  return new_pieces


def encode_ids(sp_model, text, sample=False):
  """Segments text and return token ids.

  This method is used together with sentence piece tokenizer and is forked from:
  https://github.com/google-research/google-research/blob/master/albert/tokenization.py

  Args:
    sp_model: A spm.SentencePieceProcessor object.
    text: The input text to be segemented.
    sample: Whether to randomly sample a segmentation output or return a
      deterministic one.

  Returns:
    A list of token ids.
  """
  pieces = encode_pieces(sp_model, text, sample=sample)
  ids = [sp_model.PieceToId(piece) for piece in pieces]
  return ids


class FullSentencePieceTokenizer(object):
  """Runs end-to-end sentence piece tokenization.

  The interface of this class is intended to keep the same as above
  `FullTokenizer` class for easier usage.
  """

  def __init__(self, sp_model_file):
    """Inits FullSentencePieceTokenizer.

    Args:
      sp_model_file: The path to the sentence piece model file.
    """
    self.sp_model = spm.SentencePieceProcessor()
    self.sp_model.Load(sp_model_file)
    self.vocab = {
        self.sp_model.IdToPiece(i): i
        for i in six.moves.range(self.sp_model.GetPieceSize())
    }

  def tokenize(self, text):
    """Tokenizes text into pieces."""
    return encode_pieces(self.sp_model, text)

  def convert_tokens_to_ids(self, tokens):
    """Converts a list of tokens to a list of ids."""
    return [self.sp_model.PieceToId(printable_text(token)) for token in tokens]

  def convert_ids_to_tokens(self, ids):
    """Converts a list of ids ot a list of tokens."""
    return [self.sp_model.IdToPiece(id_) for id_ in ids]


In [45]:
import gc
import re
import string
import operator
from collections import defaultdict

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import STOPWORDS

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

SEED = 1337

In [46]:
def clean(tex): 
    tex = tex.lower()
    #clear at all
    
    # Urls
    tex = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tex)
    tex = re.sub(r"http?:\/\/t.co\/[A-Za-z0-9]+", "", tex)
        
    # Words with punctuations and special characters
    punctuations = '@#!?+&*[]-%.—:,/();$=><|{}^' + '\"\'\’\‘\“\”'
    for p in punctuations:
        tex = tex.replace(p, ' ')
        
    # ... and ..
    tex = tex.replace('...', ' ... ')
    if '...' not in tex:
        tex = tex.replace('..', ' ... ')      
   
    tex = re.sub('','',tex)
    tex = re.sub(' didn ’ t ',' did not ',tex)
    tex = re.sub(' don t ',' do not ',tex)
    tex = re.sub('&',' and ',tex)
    
    tex = re.sub(' $ ',' dollar ',tex)
    tex = re.sub(' fbi ', ' federal bureau of investigation ', tex)
    tex = re.sub('w /','',tex)
    tex = re.sub('\"\'\’\‘\‘', '\'', tex)
    tex = re.sub(' u . s . ',' united states ',tex)
    tex = re.sub(' u s ',' united states ',tex)
    tex = re.sub(' m . i . t . ',' massachusetts institute of technology ',tex)
    tex = re.sub(' mit ',' massachusetts institute of technology ',tex)
    tex = re.sub(' n . y . u ',' new york university ',tex)
    tex = re.sub(' nyu ',' new york university ',tex)
    tex = re.sub(' eu ',' european union ',tex)
    tex = re.sub(' e . u . ',' european union ',tex)
    tex = re.sub('>','',tex)
    tex = re.sub('<','',tex)
    tex = re.sub('%',' percent ',tex)
    tex = re.sub(' nyc ',' new york city ',tex)
    tex = re.sub(' s ',' ',tex)
    
    tex = re.sub(' january ',' MONTH ',tex)
    tex = re.sub(' february ',' MONTH ',tex)
    tex = re.sub(' march ',' MONTH ',tex)
    tex = re.sub(' april ',' MONTH ',tex)
    tex = re.sub(' may ',' MONTH ',tex)
    tex = re.sub(' june ',' MONTH ',tex)
    tex = re.sub(' july ',' MONTH ',tex)
    tex = re.sub(' august ',' MONTH ',tex)
    tex = re.sub(' september ',' MONTH ',tex)
    tex = re.sub(' october ',' MONTH ',tex)
    tex = re.sub(' november ',' MONTH ',tex)
    tex = re.sub(' december ',' MONTH ',tex)
    
    tex = re.sub(' the ',' ',tex)
    tex = re.sub(' a ',' ',tex)
    tex = re.sub(' t ',' ',tex)
    tex = re.sub(' to ',' ',tex)
    tex = re.sub(' of ',' ',tex)
    tex = re.sub(' be ',' ',tex)
    tex = re.sub(' in ',' ',tex)
    
    tex = re.sub('  ',' ',tex)
    tex = re.sub('  ',' ',tex)
    tex = re.sub('  ',' ',tex)
    
    l = []
    for i in tex.split():
        if i.isdigit():
            l.append('NUM')
        else:
            l.append(i)

    tex = ' '.join(l)
    
    tex = re.sub(' MONTH NUM NUM ',' DATE ',tex)
    tex = re.sub(' NUM NUM NUM ',' DATE ',tex)
    
    return tex

# X_title_cleaned = X_title.apply(lambda s : clean(s))
# X_text_cleaned = X_text.apply(lambda s : clean(s))


In [47]:
class ClassificationReport(Callback):
    
    def __init__(self, train_data=(), validation_data=()):
        super(Callback, self).__init__()
        
        self.X_train, self.y_train = train_data
        self.train_precision_scores = []
        self.train_recall_scores = []
        self.train_f1_scores = []
        
        self.X_val, self.y_val = validation_data
        self.val_precision_scores = []
        self.val_recall_scores = []
        self.val_f1_scores = [] 
               
    def on_epoch_end(self, epoch, logs={}):
        train_predictions = np.round(self.model.predict(self.X_train, verbose=0))        
        train_precision = precision_score(self.y_train, train_predictions, average='macro')
        train_recall = recall_score(self.y_train, train_predictions, average='macro')
        train_f1 = f1_score(self.y_train, train_predictions, average='macro')
        self.train_precision_scores.append(train_precision)        
        self.train_recall_scores.append(train_recall)
        self.train_f1_scores.append(train_f1)
        
        val_predictions = np.round(self.model.predict(self.X_val, verbose=0))
        val_precision = precision_score(self.y_val, val_predictions, average='macro')
        val_recall = recall_score(self.y_val, val_predictions, average='macro')
        val_f1 = f1_score(self.y_val, val_predictions, average='macro')
        self.val_precision_scores.append(val_precision)        
        self.val_recall_scores.append(val_recall)        
        self.val_f1_scores.append(val_f1)
        
        print('\nEpoch: {} - Training Precision: {:.6} - Training Recall: {:.6} - Training F1: {:.6}'.format(epoch + 1, train_precision, train_recall, train_f1))
        print('Epoch: {} - Validation Precision: {:.6} - Validation Recall: {:.6} - Validation F1: {:.6}'.format(epoch + 1, val_precision, val_recall, val_f1)) 

In [33]:
%%time
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1', trainable=True)

CPU times: user 5.48 s, sys: 1.17 s, total: 6.65 s
Wall time: 7.33 s


In [71]:
class DisasterDetector:
    
    def __init__(self, bert_layer, max_seq_length=128, lr=0.0001, epochs=15, batch_size=32):
        
        # BERT and Tokenization params
        self.bert_layer = bert_layer
        
        self.max_seq_length = max_seq_length        
        vocab_file = self.bert_layer.resolved_object.vocab_file.asset_path.numpy()
        do_lower_case = self.bert_layer.resolved_object.do_lower_case.numpy()
        self.tokenizer = FullTokenizer(vocab_file, do_lower_case)
        
        # Learning control params
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        
        self.models = []
        self.scores = {}
        
        
    def encode(self, texts):
                
        all_tokens = []
        all_masks = []
        all_segments = []

        for text in texts:
            text = self.tokenizer.tokenize(text)
            text = text[:self.max_seq_length - 2]
            input_sequence = ['[CLS]'] + text + ['[SEP]']
            pad_len = self.max_seq_length - len(input_sequence)

            tokens = self.tokenizer.convert_tokens_to_ids(input_sequence)
            tokens += [0] * pad_len
            pad_masks = [1] * len(input_sequence) + [0] * pad_len
            segment_ids = [0] * self.max_seq_length

            all_tokens.append(tokens)
            all_masks.append(pad_masks)
            all_segments.append(segment_ids)

        return np.array(all_tokens), np.array(all_masks), np.array(all_segments)
    
    
    def build_model(self):
        
        input_word_ids = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='input_word_ids')
        input_mask = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='input_mask')
        segment_ids = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='segment_ids')    
        
        pooled_output, sequence_output = self.bert_layer([input_word_ids, input_mask, segment_ids])   
        clf_output = sequence_output[:, 0, :]
        out = Dense(1, activation='sigmoid')(clf_output)
        
        model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
        optimizer = SGD(learning_rate=self.lr, momentum=0.8)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        return model
    
    
    def train(self, X):
        
#         for fold, (trn_idx, val_idx) in enumerate(skf.split(X['text_cleaned'], X['label'])):
            
#             print('\nFold {}\n'.format(fold))
            fold = 0
            trn_idx = X['text_cleaned'].index
            val_idx = [0]
            X_trn_encoded = self.encode(X.loc[trn_idx, 'text_cleaned'].str.lower())
            y_trn = X.loc[trn_idx, 'label']
            X_val_encoded = self.encode(X.loc[val_idx, 'text_cleaned'].str.lower())
            y_val = X.loc[val_idx, 'label']
        
            # Callbacks
            metrics = ClassificationReport(train_data=(X_trn_encoded, y_trn), validation_data=(X_val_encoded, y_val))
            
            # Model
            model = self.build_model()        
            model.fit(X_trn_encoded, y_trn, validation_data=(X_val_encoded, y_val), callbacks=[metrics], epochs=self.epochs, batch_size=self.batch_size)
            
            self.models.append(model)
            self.scores[fold] = {
                'train': {
                    'precision': metrics.train_precision_scores,
                    'recall': metrics.train_recall_scores,
                    'f1': metrics.train_f1_scores                    
                },
                'validation': {
                    'precision': metrics.val_precision_scores,
                    'recall': metrics.val_recall_scores,
                    'f1': metrics.val_f1_scores                    
                }
            }
                    
                
    def plot_learning_curve(self):
        
        fig, axes = plt.subplots(nrows=K, ncols=2, figsize=(20, K * 6), dpi=100)
    
        for i in range(K):
            
            # Classification Report curve
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[i].history.history['val_accuracy'], ax=axes[i][0], label='val_accuracy')
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['precision'], ax=axes[i][0], label='val_precision')
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['recall'], ax=axes[i][0], label='val_recall')
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['f1'], ax=axes[i][0], label='val_f1')        

            axes[i][0].legend() 
            axes[i][0].set_title('Fold {} Validation Classification Report'.format(i), fontsize=14)

            # Loss curve
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[0].history.history['loss'], ax=axes[i][1], label='train_loss')
            sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[0].history.history['val_loss'], ax=axes[i][1], label='val_loss')

            axes[i][1].legend() 
            axes[i][1].set_title('Fold {} Train / Validation Loss'.format(i), fontsize=14)

            for j in range(2):
                axes[i][j].set_xlabel('Epoch', size=12)
                axes[i][j].tick_params(axis='x', labelsize=12)
                axes[i][j].tick_params(axis='y', labelsize=12)

        plt.show()
        
        
    def predict(self, X):
        
        X_test_encoded = self.encode(X['text_cleaned'].str.lower())
        y_pred = np.zeros((X_test_encoded[0].shape[0], 1))

        for model in self.models:
            y_pred += model.predict(X_test_encoded) / len(self.models)

        return y_pred

In [72]:
df_train = pd.read_csv('corona.csv', index_col=0)
df_train['text_cleaned'] = (df_train['title'].fillna('') + ' ' + df_train['text'].fillna('')).apply(clean)

In [73]:
K = 1
skf = StratifiedKFold(n_splits=K, random_state=SEED, shuffle=True)

DISASTER = df_train['label'] == 1
print('Whole Training Set Shape = {}'.format(df_train.shape))

ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [ ]:
clf = DisasterDetector(bert_layer, max_seq_length=128, lr=0.0001, epochs=10, batch_size=32)
clf.train(df_train)

Train on 142 samples, validate on 1 samples
Epoch 1/10
 64/142 [============>.................] - ETA: 1:37 - loss: 0.5998 - accuracy: 0.6719

In [67]:
clf.scores

{0: {'train': {'precision': [0.38571428571428573,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718],
   'recall': [0.4909090909090909, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
   'f1': [0.432,
    0.4365079365079365,
    0.4365079365079365,
    0.4365079365079365,
    0.4365079365079365,
    0.4365079365079365,
    0.4365079365079365,
    0.4365079365079365,
    0.4365079365079365,
    0.4365079365079365]},
  'validation': {'precision': [0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718,
    0.3873239436619718],
   'recall': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
   'f1': [0.4365079365079365,
    0.4365079365079365,
    0.4365079365079365,
    0.4